# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921679113286                   -0.69    5.9         
  2   -7.926168193673       -2.35       -1.22    1.0    160ms
  3   -7.926837488093       -3.17       -2.36    1.6    177ms
  4   -7.926861477875       -4.62       -3.02    2.6    197ms
  5   -7.926861632899       -6.81       -3.35    1.9    196ms
  6   -7.926861666092       -7.48       -3.72    1.9    169ms
  7   -7.926861680358       -7.85       -4.33    1.0    152ms
  8   -7.926861681724       -8.86       -4.75    2.2    180ms
  9   -7.926861681862       -9.86       -5.37    1.6    164ms
 10   -7.926861681872      -11.02       -6.01    2.0    204ms
 11   -7.926861681872      -12.25       -6.30    2.1    183ms
 12   -7.926861681873      -12.86       -7.16    1.4    162ms
 13   -7.926861681873      -14.75       -7.95    2.5    195ms
 14   -7.926861681873      -14.27       -7.76    2.8    203ms
 15   -7.

If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921745307570                   -0.69    5.6         
  2   -7.926168085018       -2.35       -1.22    1.0    178ms
  3   -7.926835944747       -3.18       -2.37    1.6    203ms
  4   -7.926861498611       -4.59       -2.99    2.9    243ms
  5   -7.926861625844       -6.90       -3.30    2.0    194ms
  6   -7.926861664578       -7.41       -3.68    1.5    207ms
  7   -7.926861680747       -7.79       -4.43    1.5    180ms
  8   -7.926861681845       -8.96       -5.18    2.2    217ms
  9   -7.926861681862      -10.78       -5.24    2.2    222ms
 10   -7.926861681872      -10.99       -5.95    1.0    194ms
 11   -7.926861681873      -12.10       -6.97    2.1    193ms
 12   -7.926861681873      -13.46       -7.42    2.9    249ms
 13   -7.926861681873   +  -14.75       -7.54    2.2    212ms
 14   -7.926861681873      -14.45       -8.65    1.2    181ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921706776683                   -0.69    5.8         
  2   -7.926174452282       -2.35       -1.22    1.0    212ms
  3   -7.926840308768       -3.18       -2.37    1.6    198ms
  4   -7.926864909933       -4.61       -3.02    2.9    250ms
  5   -7.926865050929       -6.85       -3.36    2.0    195ms
  6   -7.926865079180       -7.55       -3.75    1.5    178ms
  7   -7.926865091600       -7.91       -4.34    1.2    192ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
